In [1]:
import sys
import os
fname = r'C:\Users\agmontesb\Documents\GitHub\excel\tests\test_base_workbook.py'
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(fname), '..')))

In [2]:
import pytest
import openpyxl as px
import pandas as pd
import itertools
import collections

from excel_workbook import (
    ExcelWorkbook, ExcelTable, 
    cell_address, cell_pattern, 
    data_in_range, tbl_address, rgn_pattern,
    XlErrors, TABLE_DATA_MAP, EMPTY_CELL, 
    tbl_pattern, rgn_pattern
    )



In [3]:
def base_workbook(case):
    # Create a base workbook for testing
    filename = r"C:\Users\agmontesb\Downloads\excel_module_test.xlsx"
    wb = px.load_workbook(filename)

    excel_wb = ExcelWorkbook('excel_module_test')

    ws_name = "No links, No parameters"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["G4:I9", "F3:I9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl1 = ExcelTable(wsheet, 'sh1_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh1_tbl1 = sh1_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["G11:H15", "F12:H15"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh1_tbl2 = ExcelTable(wsheet, 'sh1_tbl2', ws_range, fmls, values, recalc=True)
    m_sh1_tbl2 = sh1_tbl2.minimun_table()

    ws_name = "Parameters and inner links"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F4:H9", "E3:H9"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl1 = ExcelTable(wsheet, 'sh2_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl1 = sh2_tbl1.minimun_table()

    # Tabla 2
    ws_range = ["F13:H17", "E12:H17"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh2_tbl2 = ExcelTable(wsheet, 'sh2_tbl2', ws_range, fmls, values, recalc=True)
    # m_sh2_tbl2 = sh2_tbl2.minimun_table()

    ws_name = "Outer links, outer parameter"
    ws = wb[ws_name]
    wsheet = excel_wb.create_worksheet(ws_name)

    # Tabla 1
    ws_range = ["F3:H8", "E2:H8"][case]
    fmls, values = data_in_range(ws, ws_range)
    sh3_tbl1 = ExcelTable(wsheet, 'sh3_tbl1', ws_range, fmls, values, recalc=True)
    # m_sh3_tbl1 = sh3_tbl1.minimun_table()

    wb.close()
    return excel_wb


In [4]:
wb = base_workbook(1)

In [5]:
wb.sheets[1]

col,E,F,G,H
row,,,,
2,,,0,
3,tabla 1:,,,
4,l1,700,250,950
5,l2,1200,230,1430
6,l3,340,100,440
7,l4,128,85,213
8,l5,45,18,63
9,,2413,683,3096
10,,,,


In [6]:
ws = wb.sheets[1]
tbl = ws.tables[0]
df = tbl.data

In [7]:
dmy = ws.all_dependents({tbl:{f"'{ws.id}'!{tbl.data.loc['G6'].code}"}}, with_links=True)

In [15]:
list(tbl.data.code.items())

[('F4', 'A9'),
 ('F5', 'A11'),
 ('F6', 'A14'),
 ('F7', 'A15'),
 ('F8', 'A17'),
 ('F9', 'A6'),
 ('G4', 'A10'),
 ('G5', 'A12'),
 ('G6', 'A13'),
 ('G7', 'A16'),
 ('G8', 'A18'),
 ('G9', 'A7'),
 ('H4', 'A1'),
 ('H5', 'A2'),
 ('H6', 'A3'),
 ('H7', 'A4'),
 ('H8', 'A5'),
 ('H9', 'A8'),
 ('E8', 'A19'),
 ('E6', 'A20'),
 ('E3', 'A21'),
 ('E5', 'A22'),
 ('E4', 'A23'),
 ('E7', 'A24')]

In [ ]:
dmy =  [1,2]
dmy.extend({3, 4})
dmy

In [ ]:
keys = dmy.keys()
[key.title for key in keys]

In [ ]:
dmy[list(keys)[1]]

In [ ]:
tbl = list(keys)[2]
[tbl.encoder('decode', x.split('!')[-1]) for x in dmy[tbl]]

In [7]:
tbl.data_rng = 'E12:I17'

In [8]:
values = {
    'I13': '=F13+F15',
    'I14': '=I13+5',
    'I15': '=I14+10',
}
tbl.set_records(values, field='fml')

In [10]:
values = {
    'I13': '=1000+I15',
}
tbl.set_records(values, field='fml')

In [9]:
tbl.recalculate(recalc=True)

In [ ]:
tbl

In [ ]:
tbl.evaluate('=E13&" "&E16')

In [ ]:
def bb():
    answer = [1,2,3,4]
    return *answer


In [77]:
dmy ='''tabla2:			
l1	25	100	0
l2	10	25	35
l3	15	38	53
l4	80	2438	2518
	105	2501	2606'''

In [52]:
dmy= '''
tabla 1:			
l1	700	250	950
l2	1200	230	1430
l3	340	100	440
l4	128	85	213
l5	45	18	63
	2413	683	3096
'''

In [13]:
lst = [x.split('\t') for x in dmy.split('\n')] 

In [ ]:
pd.DataFrame(lst)

In [50]:
def excel_df(tbl_str, row_ini, col_ini):
    lst = [x.split('\t') for x in tbl_str.strip().split('\n')]
    ndx1, ndx2 = int(row_ini), int(row_ini) + len(lst)                 #map(int, row_rng.split(':'))
    col1, col2 = ord(col_ini), ord(col_ini) + len(lst[0])                            #map(ord, col_rng.split(':'))
    df = (
        pd.DataFrame(lst, index=pd.RangeIndex(ndx1, ndx2), columns=[chr(x) for x in range(col1, col2)])
        .unstack()
        .to_frame()
        .rename(columns={0:'value'})
        .assign(cell=lambda db: db.index.map(lambda x: x[0] + str(x[1])))
        .set_index('cell')
    )
    return df.value

In [82]:
base = excel_df(dmy, '12', 'E')

In [127]:
snew = '''tabla2:	0	20	30
l1	25	100	0
l2	10	25	35
l3	15	38	53
l4	80	2438	2518
	105	2501	2606
'''

In [128]:
vnew = excel_df(snew, '12', 'E')

In [ ]:
mask = vnew != base
str(vnew[mask].to_dict()).replace('\n', '')

In [106]:
code = '''{'F13': '3',
 'G14': '41',
 'G16': '2416',
 'G17': '2495',
 'H14': '51',
 'H16': '2496',
 'H17': '2600'}'''

In [ ]:
code.replace('\n', '')

In [ ]:
linf, lsup, cell

In [ ]:
linf <= cell <= lsup